In [ ]:
import numpy as np
import torch
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import pygmtools as pygm
pygm.set_backend('pytorch')

from torch.utils.data import DataLoader

import wandb

from datasets.graph_matching_dataset_our_algorithm import GraphMatchingDataset

from sklearn.model_selection import train_test_split
from collections import defaultdict
from torch.utils.data import Subset

from visualization_tools.vvg_loader import vvg_to_df
from spatial_vessel_matching import spatial_vessel_matching
from tools.calculate_metrics import calculate_metrics

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import pandas as pd

classif = 'pygm_classic'
models = ['ipfp', 'rrwm', 'sm']

for model in models:
    print('------------------------')
    print(f"Model: {model}")

    # Read the CSV file
    df = pd.read_csv(f'GM_results/{classif}/{model}_evaluation_metrics.csv')

    # Define regex patterns to filter indices for each range
    patterns = {
        '0_to_19': r"_([0-1]?[0-9])\b",      # Matches _0 to _19
        '20_to_39': r"_(2[0-9]|3[0-9])\b",    # Matches _20 to _39
        '40_to_59': r"_(4[0-9]|5[0-9])\b"     # Matches _40 to _59
    }

    augmentations = {
        '0_to_19': "translation",      
        '20_to_39': "rotation",   
        '40_to_59': "elastic deformation" 
    }

    # Create an empty DataFrame to store results
    results_df = pd.DataFrame()

    # Loop through each pattern and compute mean and std for each range
    for range_name, pattern in patterns.items():
        # Filter the dataframe based on the pattern
        filtered_df = df[df['index'].str.contains(pattern)]
        
        # Calculate the mean and standard deviation of the metrics columns
        results = filtered_df[['accuracy', 'precision', 'recall', 'f1']].agg(['mean', 'std'])
        
        # Add a range identifier to the results
        results['range'] = range_name
        results['augmentation'] = augmentations[range_name]
        
        # Append the results to the final DataFrame
        results_df = pd.concat([results_df, results])

    # Save the results to a CSV file
    results_df.to_csv(f'GM_results/{classif}/{model}_augmentations_metrics_summary.csv')

    # Display the results
    print(results_df)




------------------------
Model: ipfp
      accuracy  precision    recall        f1     range         augmentation
mean  0.883297   0.887384  0.994451  0.935805   0_to_19          translation
std   0.084763   0.083583  0.009399  0.049735   0_to_19          translation
mean  0.728858   0.734014  0.990191  0.835465  20_to_39             rotation
std   0.140858   0.141547  0.017759  0.093869  20_to_39             rotation
mean  0.701341   0.711654  0.971836  0.813113  40_to_59  elastic deformation
std   0.162209   0.158789  0.039208  0.113764  40_to_59  elastic deformation
------------------------
Model: rrwm
      accuracy  precision    recall        f1     range         augmentation
mean  0.873552   0.877508  0.994426  0.930291   0_to_19          translation
std   0.084101   0.082704  0.009529  0.049375   0_to_19          translation
mean  0.724783   0.729888  0.990107  0.832862  20_to_39             rotation
std   0.138866   0.139626  0.017888  0.093289  20_to_39             rotation
me

/var/folders/sz/_bwt8hh13glfdzmsxchrwd7c0000gn/T/ipykernel_79195/710127376.py:32: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df = df[df['index'].str.contains(pattern)]
/var/folders/sz/_bwt8hh13glfdzmsxchrwd7c0000gn/T/ipykernel_79195/710127376.py:32: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df = df[df['index'].str.contains(pattern)]
/var/folders/sz/_bwt8hh13glfdzmsxchrwd7c0000gn/T/ipykernel_79195/710127376.py:32: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df = df[df['index'].str.contains(pattern)]
/var/folders/sz/_bwt8hh13glfdzmsxchrwd7c0000gn/T/ipykernel_79195/710127376.py:32: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the gro

In [26]:
import pandas as pd

classif = 'baseline'
models = ['baseline']

for model in models:
    print('------------------------')
    print(f"Model: {model}")

    # Read the CSV file
    df = pd.read_csv(f'GM_results/{classif}/{model}_evaluation_metrics.csv')

    # Define regex patterns to filter indices for each range
    patterns = {
        '0_to_19': r"_([0-1]?[0-9])\b",      # Matches _0 to _19
        '20_to_39': r"_(2[0-9]|3[0-9])\b",    # Matches _20 to _39
        '40_to_59': r"_(4[0-9]|5[0-9])\b"     # Matches _40 to _59
    }

    augmentations = {
        '0_to_19': "translation",      
        '20_to_39': "rotation",   
        '40_to_59': "elastic deformation" 
    }

    # Create an empty DataFrame to store results
    results_df = pd.DataFrame()

    # Loop through each pattern and compute mean and std for each range
    for range_name, pattern in patterns.items():
        # Filter the dataframe based on the pattern
        filtered_df = df[df['index'].str.contains(pattern)]
        
        # Calculate the mean and standard deviation of the metrics columns
        results = filtered_df[['accuracy', 'precision', 'recall', 'f1']].agg(['mean', 'std'])
        
        # Add a range identifier to the results
        results['range'] = range_name
        results['augmentation'] = augmentations[range_name]
        
        # Append the results to the final DataFrame
        results_df = pd.concat([results_df, results])

    # Save the results to a CSV file
    results_df.to_csv(f'GM_results/{classif}/{model}_augmentations_metrics_summary.csv')

    # Display the results
    print(results_df)




------------------------
Model: baseline
      accuracy  precision    recall        f1     range         augmentation
mean  0.900920   0.902431  0.997615  0.944417   0_to_19          translation
std   0.103003   0.102610  0.006196  0.061786   0_to_19          translation
mean  0.861037   0.862295  0.997254  0.922170  20_to_39             rotation
std   0.092777   0.093144  0.006459  0.055128  20_to_39             rotation
mean  0.828468   0.830400  0.991853  0.898492  40_to_59  elastic deformation
std   0.130175   0.130629  0.013070  0.083155  40_to_59  elastic deformation


/var/folders/sz/_bwt8hh13glfdzmsxchrwd7c0000gn/T/ipykernel_79195/3891710026.py:32: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df = df[df['index'].str.contains(pattern)]
/var/folders/sz/_bwt8hh13glfdzmsxchrwd7c0000gn/T/ipykernel_79195/3891710026.py:32: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df = df[df['index'].str.contains(pattern)]
/var/folders/sz/_bwt8hh13glfdzmsxchrwd7c0000gn/T/ipykernel_79195/3891710026.py:32: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df = df[df['index'].str.contains(pattern)]


In [29]:
import pandas as pd

classif = 'pygm_neural'
models = ['cie', 'ngm'] # 'ipca_gm', 'pca_gm'

for model in models:
    print('------------------------')
    print(f"Model: {model}")

    # Read the CSV file
    df = pd.read_csv(f'GM_results/{classif}/{model}_evaluation_metrics.csv')

    # Define regex patterns to filter indices for each range
    patterns = {
        '0_to_19': r"_([0-1]?[0-9])\b",      # Matches _0 to _19
        '20_to_39': r"_(2[0-9]|3[0-9])\b",    # Matches _20 to _39
        '40_to_59': r"_(4[0-9]|5[0-9])\b"     # Matches _40 to _59
    }

    augmentations = {
        '0_to_19': "translation",      
        '20_to_39': "rotation",   
        '40_to_59': "elastic deformation" 
    }

    # Create an empty DataFrame to store results
    results_df = pd.DataFrame()

    # Loop through each pattern and compute mean and std for each range
    for range_name, pattern in patterns.items():
        # Filter the dataframe based on the pattern
        filtered_df = df[df['index'].str.contains(pattern)]
        
        # Calculate the mean and standard deviation of the metrics columns
        results = filtered_df[['accuracy', 'precision', 'recall', 'f1']].agg(['mean', 'std'])
        
        # Add a range identifier to the results
        results['range'] = range_name
        results['augmentation'] = augmentations[range_name]
        
        # Append the results to the final DataFrame
        results_df = pd.concat([results_df, results])

    # Save the results to a CSV file
    results_df.to_csv(f'GM_results/{classif}/{model}_augmentations_metrics_summary.csv')

    # Display the results
    print(results_df)




------------------------
Model: cie
      accuracy  precision    recall        f1     range         augmentation
mean  0.962984   0.965256  0.997364  0.980801   0_to_19          translation
std   0.033239   0.031521  0.005908  0.017655   0_to_19          translation
mean  0.927810   0.929976  0.997059  0.961707  20_to_39             rotation
std   0.048089   0.047806  0.006365  0.026352  20_to_39             rotation
mean  0.894319   0.899286  0.990368  0.940764  40_to_59  elastic deformation
std   0.088052   0.085188  0.015874  0.052798  40_to_59  elastic deformation
------------------------
Model: ngm
      accuracy  precision    recall        f1     range         augmentation
mean  0.943659   0.946128  0.996938  0.970288   0_to_19          translation
std   0.048767   0.047310  0.006307  0.026681   0_to_19          translation
mean  0.901314   0.904536  0.995486  0.946642  20_to_39             rotation
std   0.064303   0.064020  0.009210  0.036437  20_to_39             rotation
mean

/var/folders/sz/_bwt8hh13glfdzmsxchrwd7c0000gn/T/ipykernel_79195/2166405234.py:32: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df = df[df['index'].str.contains(pattern)]
/var/folders/sz/_bwt8hh13glfdzmsxchrwd7c0000gn/T/ipykernel_79195/2166405234.py:32: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df = df[df['index'].str.contains(pattern)]
/var/folders/sz/_bwt8hh13glfdzmsxchrwd7c0000gn/T/ipykernel_79195/2166405234.py:32: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df = df[df['index'].str.contains(pattern)]
/var/folders/sz/_bwt8hh13glfdzmsxchrwd7c0000gn/T/ipykernel_79195/2166405234.py:32: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the